In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../BigQuery/letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


# Clean track info
Only Retain the information we need.
- id
- name
- explicit : 1 for explicit lyrics and 0 for not
- available_markets : number of available markets
- popularity

In [2]:
track_info = pd.read_csv("track_info info.csv")

In [3]:
track_info.loc[0,'available_markets'].replace('[','').replace(']','').replace("'",'').replace(' ', '').split(',')
# track_info = tracks[['id', 'name', 'popularity', '']]

['AR',
 'AU',
 'AT',
 'BE',
 'BO',
 'BR',
 'BG',
 'CA',
 'CL',
 'CO',
 'CR',
 'CY',
 'CZ',
 'DK',
 'DO',
 'DE',
 'EC',
 'EE',
 'SV',
 'FI',
 'FR',
 'GR',
 'GT',
 'HN',
 'HK',
 'HU',
 'IS',
 'IE',
 'IT',
 'LV',
 'LT',
 'LU',
 'MY',
 'MT',
 'MX',
 'NL',
 'NZ',
 'NI',
 'NO',
 'PA',
 'PY',
 'PE',
 'PH',
 'PL',
 'PT',
 'SG',
 'SK',
 'ES',
 'SE',
 'CH',
 'TW',
 'TR',
 'UY',
 'US',
 'GB',
 'AD',
 'LI',
 'MC',
 'ID',
 'JP',
 'TH',
 'VN',
 'RO',
 'IL',
 'ZA',
 'SA',
 'AE',
 'BH',
 'QA',
 'OM',
 'KW',
 'EG',
 'MA',
 'DZ',
 'TN',
 'LB',
 'JO',
 'PS',
 'IN',
 'BY',
 'KZ',
 'MD',
 'UA',
 'AL',
 'BA',
 'HR',
 'ME',
 'MK',
 'RS',
 'SI',
 'KR',
 'BD',
 'PK',
 'LK',
 'GH',
 'KE',
 'NG',
 'TZ',
 'UG',
 'AG',
 'AM',
 'BS',
 'BB',
 'BZ',
 'BT',
 'BW',
 'BF',
 'CV',
 'CW',
 'DM',
 'FJ',
 'GM',
 'GE',
 'GD',
 'GW',
 'GY',
 'HT',
 'JM',
 'KI',
 'LS',
 'LR',
 'MW',
 'MV',
 'ML',
 'MH',
 'FM',
 'NA',
 'NR',
 'NE',
 'PW',
 'PG',
 'WS',
 'SM',
 'ST',
 'SN',
 'SC',
 'SL',
 'SB',
 'KN',
 'LC',
 'VC',
 'SR',
 'TL',

In [5]:
def compute_markets(s):
    markets = s.replace('[','').replace(']','').replace("'",'').replace(' ', '').split(',')
    if markets[0] == '':
        return 0
    else:
        return len(markets)

In [6]:
cat = pd.unique(track_info['explicit'])
track_info['num_markets'] = track_info['available_markets'].apply(compute_markets)
track_info['explicit_trans'] = None 
track_info['explicit_trans'][track_info['explicit'] == cat[0]] = 0
track_info['explicit_trans'][track_info['explicit'] == cat[1]] = 1
track_info = track_info[['id','name','popularity','explicit_trans','num_markets']]
track_info.rename(columns={'num_markets':'available_markets', 'explicit_trans':'explicit'}, inplace=True)

In [9]:
track_info[["id", "name", "popularity", "explicit", "available_markets"]].to_csv("track clear info.csv")

# Load track clear info to big query

In [15]:
dataset_id = "snappy-boulder-378707.TrackClearInfo"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, timeout=30)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset snappy-boulder-378707.TrackClearInfo


In [16]:
schema = [
    bigquery.SchemaField("unnamed","INTEGER"),
    bigquery.SchemaField("id","STRING"),
    bigquery.SchemaField("name","STRING"),
    bigquery.SchemaField("popularity","INTEGER"),
    bigquery.SchemaField("explicit","INTEGER"),
    bigquery.SchemaField("available_markets","INTEGER"),
]


# TODO(developer): Set table_id to the ID of the table to create.
table_id = "snappy-boulder-378707.TrackClearInfo.TrackClearInfo"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, timeout=30)  # Make an API request.

In [17]:
job_config = bigquery.LoadJobConfig(
    schema = schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

file_path = './track clear info.csv'
table_id = "snappy-boulder-378707.TrackClearInfo.TrackClearInfo"

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 74925 rows and 6 columns to snappy-boulder-378707.TrackClearInfo.TrackClearInfo


In [20]:
%%bigquery
ALTER TABLE snappy-boulder-378707.TrackClearInfo.TrackClearInfo
DROP COLUMN unnamed

Query is running:   0%|          |

""


In [21]:
%%bigquery
ALTER TABLE snappy-boulder-378707.TrackClearInfo.TrackClearInfo
ADD PRIMARY KEY (id) NOT ENFORCED

Query is running:   0%|          |

""


# Clear Audio features

In [25]:

schema = [
    bigquery.SchemaField("unnamed","INTEGER"),
    bigquery.SchemaField("danceability","FLOAT"),
    bigquery.SchemaField("energy","FLOAT"),
    bigquery.SchemaField("key","INTEGER"),
    bigquery.SchemaField("loudness","FLOAT"),
    bigquery.SchemaField("mode","INTEGER"),
    bigquery.SchemaField("speechiness","FLOAT"),
    bigquery.SchemaField("acousticness","FLOAT"),
    bigquery.SchemaField("instrumentalness","FLOAT"),
    bigquery.SchemaField("liveness","FLOAT"),
    bigquery.SchemaField("valence","FLOAT"),
    bigquery.SchemaField("tempo","FLOAT"),
    bigquery.SchemaField("type","STRING"),
    bigquery.SchemaField("id","STRING"),
    bigquery.SchemaField("uri","STRING"),
    bigquery.SchemaField("track_href","STRING"),
    bigquery.SchemaField("analysis_url","STRING"),
    bigquery.SchemaField("duration_ms","INTEGER"),
    bigquery.SchemaField("time_signature","INTEGER"),
]

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "snappy-boulder-378707.AudioFeatures.AudioFeatures"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table, timeout=30)  # Make an API request.
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Created table snappy-boulder-378707.AudioFeatures.AudioFeatures


In [26]:
job_config = bigquery.LoadJobConfig(
    schema = schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)


file_path = './audio features.csv'
table_id = "snappy-boulder-378707.AudioFeatures.AudioFeatures"

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 74856 rows and 19 columns to snappy-boulder-378707.AudioFeatures.AudioFeatures


In [34]:
%%bigquery
ALTER TABLE snappy-boulder-378707.AudioFeatures.AudioFeatures
DROP COLUMN unnamed, drop column type, drop column uri,drop column track_href,drop column analysis_url;

Query is running:   0%|          |

""


In [35]:
%%bigquery
ALTER TABLE snappy-boulder-378707.AudioFeatures.AudioFeatures
ADD PRIMARY KEY (id) NOT ENFORCED

Query is running:   0%|          |

""
